In [1]:
import os
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

os.environ["SPARK_HOME"] = "/usr/local/spark"
os.environ["PYSPARK_PYTHON"] = "/home/pigidser/anaconda3/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python3"
os.environ["PYSPARK_SUBMIT_ARGS"] = "pyspark-shell"

spark = SparkSession.builder.master("local").appName("spark_test").getOrCreate()

In [2]:
# load dataframe
cdf = spark.read.format("csv") \
    .option("mode", "FAILFAST") \
    .option("inferSchema", "true") \
    .option("header","true") \
    .option("path", "countries_of_the_world.csv") \
    .load()

In [13]:
cdf.select(F.max("Population").alias("MaxPopulation")).show()

+-------------+
|MaxPopulation|
+-------------+
|   1313973713|
+-------------+



In [14]:
# groupBy
# Посмотрим агрегат в разбивке по колонке (используем groupBy())

cdf.groupBy("Region").max("Population").show()

+--------------------+---------------+
|              Region|max(Population)|
+--------------------+---------------+
|BALTICS          ...|        3585906|
|C.W. OF IND. STATES |      142893540|
|ASIA (EX. NEAR EA...|     1313973713|
|WESTERN EUROPE   ...|       82422299|
|NORTHERN AMERICA ...|      298444215|
|NEAR EAST        ...|       70413958|
|EASTERN EUROPE   ...|       38536869|
|OCEANIA          ...|       20264082|
|SUB-SAHARAN AFRIC...|      131859731|
|NORTHERN AFRICA  ...|       78887007|
|LATIN AMER. & CAR...|      188078227|
+--------------------+---------------+



In [15]:
# Оконная функция
# Сделаем то же самое, но с использованием оконной функции.

from pyspark.sql.window import Window

windowSpec = Window.partitionBy("Region")
cdf.select("Region",F.max("Population").over(windowSpec).alias("MaxPopulation")).distinct().show()

+--------------------+-------------+
|              Region|MaxPopulation|
+--------------------+-------------+
|BALTICS          ...|      3585906|
|C.W. OF IND. STATES |    142893540|
|ASIA (EX. NEAR EA...|   1313973713|
|WESTERN EUROPE   ...|     82422299|
|NORTHERN AMERICA ...|    298444215|
|NEAR EAST        ...|     70413958|
|EASTERN EUROPE   ...|     38536869|
|OCEANIA          ...|     20264082|
|SUB-SAHARAN AFRIC...|    131859731|
|NORTHERN AFRICA  ...|     78887007|
|LATIN AMER. & CAR...|    188078227|
+--------------------+-------------+



In [18]:
# Сортировка
# Упорядочим предыдущий результат по убыванию максимального населения

cdf.select("Region",F.max("Population").over(windowSpec).alias("MaxPopulation"))\
    .distinct().sort(F.desc("MaxPopulation")).show()

+--------------------+-------------+
|              Region|MaxPopulation|
+--------------------+-------------+
|ASIA (EX. NEAR EA...|   1313973713|
|NORTHERN AMERICA ...|    298444215|
|LATIN AMER. & CAR...|    188078227|
|C.W. OF IND. STATES |    142893540|
|SUB-SAHARAN AFRIC...|    131859731|
|WESTERN EUROPE   ...|     82422299|
|NORTHERN AFRICA  ...|     78887007|
|NEAR EAST        ...|     70413958|
|EASTERN EUROPE   ...|     38536869|
|OCEANIA          ...|     20264082|
|BALTICS          ...|      3585906|
+--------------------+-------------+



In [22]:
# Минимум или максимум
# найдем страны с минимальным и максимальным населением в регионах
# Комментарии:
# - оконная функция просто группирует по региону
# - добавляем колонки с минимальным и максимальным населением по региону (minp, maxp)
# - оставляем в датафрейме только страны, население которых самое маленькое или большое (используем expr)
# - добавляем колонку which, в которую записываем - какая это страна (с минимальным или максимальным населением, используем функции when().otherwise()
# - удалим неинтересные нам колонки (select())
# - упорядочим - покажем сначала самые большие страны, потом - самые маленькие (по убыванию населения)

wr = Window.partitionBy('Region')
cdf.withColumn('minp', F.min('Population').over(wr))\
    .withColumn('maxp', F.max('Population').over(wr))\
    .where((F.expr('Population==minp or Population==maxp')))\
    .withColumn("which",F.when(F.expr("Population==minp"),'MIN').otherwise('MAX'))\
    .select("Country","Region","Population","which")\
    .sort("which",F.desc("Population"))\
    .show(50, truncate=False)

+---------------------+-----------------------------------+----------+-----+
|Country              |Region                             |Population|which|
+---------------------+-----------------------------------+----------+-----+
|China                |ASIA (EX. NEAR EAST)               |1313973713|MAX  |
|United States        |NORTHERN AMERICA                   |298444215 |MAX  |
|Brazil               |LATIN AMER. & CARIB                |188078227 |MAX  |
|Russia               |C.W. OF IND. STATES                |142893540 |MAX  |
|Nigeria              |SUB-SAHARAN AFRICA                 |131859731 |MAX  |
|Germany              |WESTERN EUROPE                     |82422299  |MAX  |
|Egypt                |NORTHERN AFRICA                    |78887007  |MAX  |
|Turkey               |NEAR EAST                          |70413958  |MAX  |
|Poland               |EASTERN EUROPE                     |38536869  |MAX  |
|Australia            |OCEANIA                            |20264082  |MAX  |

In [23]:
spark.stop()